- Limpiar punctuacion
- Limpiar links

In [118]:
import pandas as pd
import nltk  
import numpy as np  
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import re
import random

In [2]:
df = pd.read_csv('todes.csv')

 
## Tweet generation with LSTM
Example in this [link](https://towardsdatascience.com/tweet-generation-with-neural-networks-lstm-and-gpt-2-e163bfd3fbd8)
 1. **Removed tweets whose lengths were less than 60 characters**. 

    Removing tweets that fall under a certain length, it removes tweets that are one word and the empty tweets that made it in the CSV file
   
   
2.  **Tweets were divided into sequences of equal lengths and placed into a list called sentences.**
 
    These sentences will be used as training examples for the model.
 
 
 3. **Vectorization**
 
    Split the tweets into x and y evenly.**x** is a 3D matrix that has the shape of the total number of sentences, length of steps(40), and the number of unique characters. **y** is a 2D matrix that has the total length of sentences and unique characters and the purpose of this vector is to retrieve the next character that’s after a sentence given by x.
 
 
 4. **Model**

In [ ]:
model = Sequential()

model.add(LSTM(len(chars) * 5, input_shape=(SEQUENCE_LENGTH, len(chars))))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars) * 2))
model.add(BatchNormalization())
model.add(Activation('selu'))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X, y, validation_split=0.05, batch_size=124, epochs=4, shuffle=True)

5. **Training the model**

      After 10 epoch the model stopped learning with a loss of around 1.5 for each of the datasets. During training, the model outputs the next character based on the probability distribution. This is known as diversity. Diversity is a value that determines how random the generated sentence is. For training purposes, we set the diversity to 0.2, 0.5, 0.7, 1.0, and 1.2 in order to visualize which diversity generated the most meaningful tweets.
  
  
6.  **Generate new tweets for the user**

    The generate_tweets function takes a random seed, which is a random tweet from the lists of tweets, and a set diversity. We chose a diversity of 0.7 because it formulated the most comprehensible tweets compared to the others. After receiving a random seed and a diversity value, the function cuts the seed length down to 40 characters. This is done because it would be benign to use an already formulated tweet to create another and because once a new tweet is generated, it would exceed the max length of characters that are permitted on twitter.

## ANALYSIS

### Dictionary to analyze the results

In [4]:
text = ''
for tweet in df.full_text:
    text += tweet
    
text = text.lower()
text = text.replace('\n','')

In [5]:
corpus = nltk.sent_tokenize(text)

In [6]:
wordfreq = {}

for sentence in corpus:
    print(sentence)
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

vimos en prime time cómo sacaban a una momia por “dignidad a las víctimas del franquismo”, pero no vamos a poder ver una misa en memoria de las víctimas del covid.
radio televisión espantosa, la televisión de todas, todos y todes @rtvegerardo morales , el inmoral, le quitó la ambulancia al hospital de humahuaca en plena pandemia.
¿es con todes?
¿con los genocidas también?
repudio a morales, a su abuso de poder y a quienes deberían frenar el etnocidio que está llevando a cabo y miran para otro lado.
https://t.co/vm433h05bb🔊🔊💜 iniciamos en vivolas jornadas ecofeministasantiextractivistaspor el buen vivir 💜🔊🔊ytb ☞ https://t.co/mfhhurdddxfcbk ☞ https://t.co/oieedmf55sy ☞ https://t.co/cyy7sfq9rp🗣️🌊✊✊ bienvenidas todas todos y todes a escuchar vamos a terminar con el patriarcado!sentimos #orgullo y queremos contarlo@espanawiphala participó de la #manifestacionvirtuallgtbi en #españaluchamos por una sociedad igualitaria, sin discriminación, que defienda los derechos de todes.🌈✊🏽🏳️‍🌈🙌el evento

(te hablo a ti, persona antiderechos)biopolímeros y cirugías para todes.
https://t.co/88bh5gnlrd@sodacaj jajajajajaj boluda los votos.
amo a la gente que me sigue.
somos todes psiquiatricosdsps d esto todes nos hacemos fans de corte y confección?
necesito otro reality para mantenerme "cuerdo" durante lo q reste d cuarentena@cocinatelefe todes queremos que damián, pero es difícil porque compite con la administradora petrolera que era pastelera profesional en programas de tv argentinos y ya había hecho muchas de las tortas que en #bakeoffargentina decía no saber hacer!!!!
@julimoret la viveza criolla todes la hemos adoptado de chiquitos jajaj me encantócuando empiezo a encariñarme con alguien after saying “todes mienten” https://t.co/bnx3lyvzfny si nos quedamos acá decidiendo entre todos todas y todes que podemos mirar a partir del próximo domingo y nos juntamos acá a debatir el programa 🤔que está pasanda en el mundo .. todes buscando un lugar, pero en lugar de buscarlo desde un lugar de

In [7]:
wordfreq

{'vimos': 5,
 'en': 598,
 'prime': 4,
 'time': 4,
 'cómo': 16,
 'sacaban': 4,
 'a': 1456,
 'una': 129,
 'momia': 4,
 'por': 472,
 '“': 34,
 'dignidad': 4,
 'las': 263,
 'víctimas': 12,
 'del': 126,
 'franquismo': 4,
 '”': 32,
 ',': 1341,
 'pero': 187,
 'no': 601,
 'vamos': 31,
 'poder': 19,
 'ver': 59,
 'misa': 4,
 'memoria': 4,
 'de': 1543,
 'covid': 14,
 '.': 809,
 'radio': 4,
 'televisión': 10,
 'espantosa': 4,
 'la': 955,
 'todas': 89,
 'todos': 118,
 'y': 1263,
 'todes': 1610,
 '@': 981,
 'rtvegerardo': 1,
 'morales': 8,
 'el': 850,
 'inmoral': 4,
 'le': 90,
 'quitó': 4,
 'ambulancia': 4,
 'al': 153,
 'hospital': 11,
 'humahuaca': 4,
 'plena': 5,
 'pandemia': 31,
 '¿es': 4,
 'con': 339,
 '?': 461,
 '¿con': 4,
 'los': 336,
 'genocidas': 4,
 'también': 47,
 'repudio': 4,
 'su': 83,
 'abuso': 8,
 'quienes': 22,
 'deberían': 10,
 'frenar': 4,
 'etnocidio': 4,
 'que': 1557,
 'está': 42,
 'llevando': 4,
 'cabo': 5,
 'miran': 6,
 'para': 473,
 'otro': 11,
 'lado': 30,
 'https': 716,
 ':'

In [8]:
from collections import OrderedDict
d_descending = OrderedDict(sorted(wordfreq.items(), key=lambda kv: kv[1], reverse=True))
d_descending

OrderedDict([('todes', 1610),
             ('que', 1557),
             ('de', 1543),
             ('a', 1456),
             (',', 1341),
             ('y', 1263),
             ('@', 981),
             ('la', 955),
             (':', 897),
             ('el', 850),
             ('.', 809),
             ('#', 734),
             ('https', 716),
             ('no', 601),
             ('en', 598),
             ('!', 510),
             ('para', 473),
             ('por', 472),
             ('es', 464),
             ('?', 461),
             ('con', 339),
             ('los', 336),
             ('un', 328),
             ('lo', 327),
             ('se', 273),
             ('me', 264),
             ('las', 263),
             ('les', 261),
             ("''", 251),
             ('nos', 210),
             ('``', 199),
             ('pero', 187),
             ('hacer', 180),
             ('bakeoffargentina', 170),
             ('como', 158),
             ('al', 153),
             ('si', 140),
     

## Datacamp Course

Clean tweets text

From https://towardsdatascience.com/predicting-trump-tweets-with-a-rnn-95e7c398b18e:

In [76]:
def removeLinks(text):
    return re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)

def removeEmojis(text):
    return text.encode('ascii', 'ignore').decode('ascii')

def removeEllipsis(text):
    return re.sub('\.\.[\.]*', " ", text)

def removeParens(text):
    return re.sub("[\(\[].*?[\)\]]", "", text)

In [81]:
text = removeLinks(text)
text = removeEllipsis(text)
text = removeEmojis(text)
text = removeParens(text)
text

'vimos en prime time cmo sacaban a una momia por dignidad a las vctimas del franquismo, pero no vamos a poder ver una misa en memoria de las vctimas del covid. radio televisin espantosa, la televisin de todas, todos y todes @rtvegerardo morales , el inmoral, le quit la ambulancia al hospital de humahuaca en plena pandemia. es con todes? con los genocidas tambin? repudio a morales, a su abuso de poder y a quienes deberan frenar el etnocidio que est llevando a cabo y miran para otro lado.   iniciamos en vivolas jornadas ecofeministasantiextractivistaspor el buen vivir ytb          bienvenidas todas todos y todes a escuchar vamos a terminar con el patriarcado!sentimos #orgullo y queremos contarlo@espanawiphala particip de la #manifestacionvirtuallgtbi en #espaaluchamos por una sociedad igualitaria, sin discriminacin, que defienda los derechos de todes.el evento:   #orgullo2020   nos pasa a todes quienes no entramos en ese estandar de belleza marcado por la sociedad. ya gordes, delgades, a

#### Here I start with the datacamp course guide 👇🏼

In [82]:
vocabulary = sorted(set(text))

# testing erasing some parts of the vocabulary!
# vocabulary = vocabulary[16:60]

# Print the vocabulary size
print('Vocabulary size:', len(vocabulary))
vocabulary

Vocabulary size: 58


[' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|']

In [84]:
# Dictionary to save the mapping from char to integer
char_to_idx = { char : idx for idx, char in enumerate(vocabulary) }

# Dictionary to save the mapping from integer to char
idx_to_char = { idx : char for idx, char in enumerate(vocabulary) }

# Print char_to_idx and idx_to_char
print(char_to_idx)
print(idx_to_char)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, ')': 8, '*': 9, '+': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '@': 28, '_': 29, '`': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56, '|': 57}
{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: '&', 7: "'", 8: ')', 9: '*', 10: '+', 11: ',', 12: '-', 13: '.', 14: '/', 15: '0', 16: '1', 17: '2', 18: '3', 19: '4', 20: '5', 21: '6', 22: '7', 23: '8', 24: '9', 25: ':', 26: ';', 27: '?', 28: '@', 29: '_', 30: '`', 31: 'a', 32: 'b', 33: 'c', 34: 'd', 35: 'e', 36: 'f', 37: 'g', 38: 'h', 39: 'i', 40: 'j', 41: 'k', 42: 'l', 43: 'm', 44: 'n', 45: 'o', 46: 'p', 47: 'q', 48: 'r', 49: 's', 50: 't', 51: 'u', 52: 'v', 53: 'w', 54: 'x', 5

In [85]:
maxlen =40

In [86]:
# Create empty lists for input and target dataset
input_data = []
target_data = []

# Iterate to get all substrings of length maxlen
for i in range(0, len(text) - maxlen):
    # Find the sequence of length maxlen starting at i
    input_data.append(text[i : i+maxlen])
    
    # Find the next char after this sequence 
    target_data.append(text[i+maxlen])

# Print number of sequences in input data
print('No of Sequences:', len(input_data))

No of Sequences: 207125


In [87]:
input_data

['vimos en prime time cmo sacaban a una mo',
 'imos en prime time cmo sacaban a una mom',
 'mos en prime time cmo sacaban a una momi',
 'os en prime time cmo sacaban a una momia',
 's en prime time cmo sacaban a una momia ',
 ' en prime time cmo sacaban a una momia p',
 'en prime time cmo sacaban a una momia po',
 'n prime time cmo sacaban a una momia por',
 ' prime time cmo sacaban a una momia por ',
 'prime time cmo sacaban a una momia por d',
 'rime time cmo sacaban a una momia por di',
 'ime time cmo sacaban a una momia por dig',
 'me time cmo sacaban a una momia por dign',
 'e time cmo sacaban a una momia por digni',
 ' time cmo sacaban a una momia por dignid',
 'time cmo sacaban a una momia por dignida',
 'ime cmo sacaban a una momia por dignidad',
 'me cmo sacaban a una momia por dignidad ',
 'e cmo sacaban a una momia por dignidad a',
 ' cmo sacaban a una momia por dignidad a ',
 'cmo sacaban a una momia por dignidad a l',
 'mo sacaban a una momia por dignidad a la',
 'o sacaba

In [88]:
# Create a 3-D zero vector to contain the encoded input sequences
x = np.zeros((len(input_data), maxlen, len(vocabulary)), dtype='float32')

# Create a 2-D zero vector to contain the encoded target characters
y = np.zeros((len(target_data), len(vocabulary)), dtype='float32')

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [89]:
# Iterate over the sequences
for s_idx, sequence in enumerate(input_data):
    # Iterate over all characters in the sequence
    for idx, char in enumerate(sequence):
        # Fill up vector x
        x[s_idx, idx, char_to_idx[char]] = 1    
    # Fill up vector y
    y[s_idx, char_to_idx[target_data[s_idx]]] = 1

In [90]:
# Create Sequential model 
model = tf.keras.Sequential()

# Add an LSTM layer of 128 units
model.add(layers.LSTM(128, input_shape=(maxlen, len(vocabulary))))

# Add a Dense output layer
model.add(layers.Dense(len(vocabulary), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Print model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               95744     
_________________________________________________________________
dense_2 (Dense)              (None, 58)                7482      
Total params: 103,226
Trainable params: 103,226
Non-trainable params: 0
_________________________________________________________________


In [91]:
# Fit the model
model.fit(x, y, batch_size=64, epochs=3, validation_split=0.2)

Epoch 1/3
2590/2590 [==============================] - 76s 29ms/step - loss: 2.4198 - val_loss: 2.1977
Epoch 2/3
2590/2590 [==============================] - 72s 28ms/step - loss: 2.1150 - val_loss: 2.0563
Epoch 3/3
2590/2590 [==============================] - 72s 28ms/step - loss: 1.9743 - val_loss: 1.9600


In [156]:
def generate_text(sentence, n):
    generated = sentence
#     generated += sentence
    for i in range(n):
    # Create a 3-D zero vector to contain the encoding of sentence.
        x_pred = np.zeros((1, maxlen, len(vocabulary)))
        
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_idx[char]] = 1.
            
        # Get probability distribution for the next character
        preds = model.predict(x_pred, verbose=0)[0]
        #print(preds)
    
        # Get the character with maximum probability
        next_index = np.argmax(preds)
        #next_index = random.sample(list(preds),0.7)
        next_index = random.choice(sorted(enumerate(example),
                       key=lambda x: x[1]
                       )[-15:])[0]
# write an if statement to check if most recent character generated is a space, if it is then generate randomly next character
# from -15 to -1 (from some array without array)

        next_char = idx_to_char[next_index]
    
        # Append the new character to the next input and generated text
        sentence = sentence[1:] + next_char
        generated += next_char
    
    # Print the generated text
    print(generated)


In [157]:
# Input sequence and generate text
sentence = "esa gente debe parar"
generate_text(sentence, 20)



esa gente debe pararrot eiesaipmcuerdupt


In [144]:
example

array([1.43619001e-01, 3.09601403e-03, 1.86012650e-03, 7.71214627e-03,
       9.98674586e-05, 7.28055093e-05, 6.76896743e-05, 1.16962634e-04,
       2.13351916e-04, 3.77779710e-04, 9.66791777e-05, 8.36510863e-03,
       8.72493314e-04, 5.97984344e-03, 1.41052855e-03, 1.49698963e-03,
       1.42622343e-03, 1.73081027e-03, 3.87489388e-04, 6.21355372e-04,
       3.60349281e-04, 2.86770461e-04, 9.16702498e-04, 3.28758266e-04,
       7.94419495e-04, 1.88012538e-03, 2.87854695e-04, 3.85484332e-03,
       1.22461161e-02, 2.62174057e-03, 1.74851903e-05, 1.22490332e-01,
       1.35361422e-02, 2.83862799e-02, 1.85262151e-02, 1.29976451e-01,
       3.90357804e-03, 8.26939382e-03, 1.49073042e-02, 1.20078430e-01,
       9.32709407e-03, 6.04929216e-03, 3.83987315e-02, 1.69543214e-02,
       2.12259181e-02, 5.12126982e-02, 2.96521783e-02, 6.18054438e-03,
       3.06836907e-02, 3.23542431e-02, 3.19511853e-02, 3.48651297e-02,
       9.62895993e-03, 2.83321552e-03, 2.35190894e-03, 1.06181921e-02,
      

In [149]:

y_hat = random.choice(sorted(enumerate(example),
                       key=lambda x: x[1]
                       )[-15:])[0]

In [150]:
y_hat

48